### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Supplementary indicators of the pace of transformation <br /> in 1.5°C pathways and selected sectoral studies

This notebook computes additional indicators of sectoral transformation for **Table 4.SM.1** in the Special Report.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load ``pyam`` package and other dependencies

In [ ]:
import yaml
import pyam

## Import scenario data, categorization and specifications files

The metadata file must have been generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
cats_15_no_lo = specs.pop('cats_15_no_lo')
marker= specs.pop('marker')

## Assign years of interest and downselect to scenarios of interest for this assessment

In [ ]:
table_years = [2020, 2030, 2050]
compare_year = 2010
years = [compare_year] + table_years

In [ ]:
cats.remove('Above 2C')

In [ ]:
df = sr1p5.filter(category=cats + ['no-climate-assessment'], year=years)

## Initialize a `pyam.Statistics` instance¶

In [ ]:
filters = [
    (('IAM pathways', 'no & lo os 1.5'), {'category': cats_15_no_lo}),
    (('IAM pathways', 'lo os 1.5'), {'category': ['1.5C low overshoot']}),
    (('IAM pathways', 'hi os 1.5'), {'category': ['1.5C high overshoot']}),
    (('IAM pathways', 'S1'), {'marker': ['S1']}),
    (('IAM pathways', 'S2'), {'marker': ['S2']}),
    (('IAM pathways', 'S5'), {'marker': ['S5']}),
    (('IAM pathways', 'LED'), {'marker': ['LED']}),
    (('sectoral studies', 'Löffler et al. (2017)'), {'model': ['GENeSYS-MOD 1.0']}),
    (('sectoral studies', 'IEA ETP (2017)'), {'model': ['IEA Energy Technology Perspective Model 2017']}),
    (('sectoral studies', 'IEA WEM (2017)'), {'model': ['IEA World Energy Model 2017']})
]

stats = pyam.Statistics(df=df, filters=filters, rows=True)

## Share of renewables in primary energy and electricity generation

In [ ]:
def add_stats_share(var_list, header, total, total_name, df=df, years=table_years):

    _df = df.filter(variable=var_list)
    for v in var_list:
        _df.require_variable(v, exclude_on_fail=True)
    _df.filter(exclude=False, inplace=True)

    component = (
        _df.timeseries()
        .groupby(['model', 'scenario']).sum()
    )
    share = component / total * 100
    
    for y in years:
        stats.add(share[y], header='Share of {}'.format(header),
                  subheader='in {} (%)'.format(total_name), row=y)

In [ ]:
pe = df.filter(variable='Primary Energy').timeseries()
pe.index = pe.index.droplevel([2, 3, 4])

In [ ]:
pe_re_vars = [
    'Primary Energy|Biomass',
    'Primary Energy|Non-Biomass Renewables'
]

add_stats_share(pe_re_vars, 'renewables', pe, 'primary energy')

In [ ]:
ele = df.filter(variable='Secondary Energy|Electricity').timeseries()
ele.index = ele.index.droplevel([2, 3, 4])

In [ ]:
ele_re_vars = [
    'Secondary Energy|Electricity|Biomass',
    'Secondary Energy|Electricity|Non-Biomass Renewables'
]

add_stats_share(ele_re_vars, 'renewables', ele, 'electricity generation')

## Share of fossil fuels in electricity generation

In [ ]:
ele_foss_vars = [
    'Secondary Energy|Electricity|Coal',
    'Secondary Energy|Electricity|Gas',
    'Secondary Energy|Electricity|Oil',
]

add_stats_share(ele_foss_vars, 'fossil fuels', ele, 'electricity generation')

## Energy demand in buildings (relative to 2010)

In [ ]:
bld = df.filter(variable='Final Energy|Residential and Commercial', year=years).timeseries()

In [ ]:
for y in table_years:
    stats.add(-(bld[y] / bld[compare_year] - 1) * 100,
              header='Change in energy demand in buildings',
              subheader='relative to {} (%)'.format(compare_year), row=y)

## Direct emissions reductions from buildings (relative to 2010)

In [ ]:
co2_bld = df.filter(variable='Emissions|CO2|Energy|Demand|Residential and Commercial', year=years).timeseries()

In [ ]:
for y in table_years:
    stats.add(-(co2_bld[y] / co2_bld[compare_year] - 1) * 100,
              header='Direct emissions reductions from buildings',
              subheader='relative to {} (%)'.format(compare_year), row=y)

## Share of low-carbon fuels and electricity in the transport sector

For the scenario presented by "Löffler et al. (2017)", the share of low-carbon fuels was read directly from one of the figures in the manuscript.

In [ ]:
trp = df.filter(variable='Final Energy|Transportation').timeseries()
trp.index = trp.index.droplevel([2, 3, 4])

In [ ]:
trp_low_vars = [
    'Final Energy|Transportation|Electricity',
    'Final Energy|Transportation|Hydrogen',
    'Final Energy|Transportation|Liquids|Biomass'
]
add_stats_share(trp_low_vars, 'low-carbon fuels', trp, 'transport')

In [ ]:
trp_ele_vars = ['Final Energy|Transportation|Electricity']
add_stats_share(trp_ele_vars, 'electricity', trp, 'transport')

In [ ]:
trp_bio_vars = ['Final Energy|Transportation|Liquids|Biomass']
add_stats_share(trp_bio_vars, 'biofuels', trp, 'transport')

## Industrial emissions (relative to 2010)

In [ ]:
co2_ind = df.filter(variable='Emissions|CO2|Energy|Demand|Industry', year=years).timeseries()

In [ ]:
for y in table_years:
    stats.add(-(co2_ind[y] / co2_ind[compare_year] - 1) * 100,
              header='Industrial emissions reductions',
              subheader='relative to {} (%)'.format(compare_year), row=y)

## Display summary statistics table and export as `xlsx`

In [ ]:
summary = stats.summarize(center='median', interquartile=True, custom_format='{:.2f}')
summary = (
    summary
    .swaplevel(0, 2, axis=0).swaplevel(1, 2, axis=0)
    .sort_index(axis=0, level=0, sort_remaining=False)
)
summary

In [ ]:
summary.to_excel('output/table_4.SM.1_suppl_sectoral_indicators.xlsx')